# WHOLE + MACHINE (mean of embeddings)

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from scipy.stats import hmean
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [2]:
eval_train = pd.read_csv('../processed_data/evaluation_train.csv')
eval_test = pd.read_csv('../processed_data/evaluation_test.csv')
eval_eval = pd.read_csv('../processed_data/evaluation_eval.csv')

In [3]:
with open('result_emb/train_emb.pickle', 'rb') as f:
    result_train = pickle.load(f)
f.close()

with open('result_emb/test_emb.pickle', 'rb') as f:
    result_test = pickle.load(f)
f.close()

with open('result_emb/eval_emb.pickle', 'rb') as f:
    result_eval = pickle.load(f)
f.close()

##############################################################

with open('result_emb/train_machine_emb.pickle', 'rb') as f:
    result_train_machine = pickle.load(f)
f.close()

with open('result_emb/test_machine_emb.pickle', 'rb') as f:
    result_test_machine = pickle.load(f)
f.close()

with open('result_emb/eval_machine_emb.pickle', 'rb') as f:
    result_eval_machine = pickle.load(f)
f.close()

In [4]:
def length_norm(mat):
    norm_mat = []
    for line in mat:
        temp = line / np.sqrt(sum(np.power(line, 2)))
        norm_mat.append(temp)
    norm_mat = np.array(norm_mat)
    return norm_mat

In [5]:
result_train_ln = length_norm(result_train)
result_test_ln = length_norm(result_test)
result_eval_ln = length_norm(result_eval)

result_train_ln_machine = length_norm(result_train_machine)
result_test_ln_machine = length_norm(result_test_machine)
result_eval_ln_machine = length_norm(result_eval_machine)

In [6]:
result_train_mean = result_train_ln*0.62 + result_train_ln_machine*(1-0.62)
result_test_mean = result_test_ln*0.62 + result_test_ln_machine*(1-0.62)
result_eval_mean = result_eval_ln*0.62 + result_eval_ln_machine*(1-0.62)

In [7]:
source_train = np.array(eval_train[eval_train['domain']=='source'].index)
target_train = np.array(eval_train[eval_train['domain']=='target'].index)

In [8]:
kmeans = KMeans(n_clusters=16, random_state=42).fit(result_train_mean[source_train])
centers = kmeans.cluster_centers_

In [9]:
a = np.min(1-np.dot(result_test_mean, centers.transpose()), axis=-1, keepdims=True)
b = np.min(1-np.dot(result_test_mean, result_train_mean[target_train].transpose()), axis=-1, keepdims=True)
cos = np.minimum(a, b)

eval_test['anomaly_score'] = cos

In [11]:
aucs = []
p_aucs = []
aucs_source = []
p_aucs_source = []
aucs_target = []
p_aucs_target = []

machine_list = eval_test['machine'].unique()
for machine in machine_list:
    temp = eval_test[eval_test['machine']==machine]
    true = temp['label'].values
    cos = temp['anomaly_score'].values
    auc = roc_auc_score(true, cos)
    p_auc = roc_auc_score(true, cos, max_fpr=0.1)
    aucs.append(auc)
    p_aucs.append(p_auc)
    print('AUC of ' + machine + ': ' + str(auc * 100))
    print('pAUC of ' + machine + ': ' + str(p_auc * 100))
    
    print('####################################################')
        
    temp_source = temp[temp['domain']=='source']
    true_source = temp_source['label'].values
    cos_source = temp_source['anomaly_score'].values
    auc = roc_auc_score(true_source, cos_source)
    p_auc = roc_auc_score(true_source, cos_source, max_fpr=0.1)
    aucs_source.append(auc)
    p_aucs_source.append(p_auc)
    print('AUC for source domain of ' + machine + ': ' + str(auc * 100))
    print('pAUC for source domain of ' + machine + ': ' + str(p_auc * 100))
    
    print('####################################################')
        
    temp_target = temp[temp['domain']=='target']
    true_target = temp_target['label'].values
    cos_target = temp_target['anomaly_score'].values
    auc = roc_auc_score(true_target, cos_target)
    p_auc = roc_auc_score(true_target, cos_target, max_fpr=0.1)
    aucs_target.append(auc)
    p_aucs_target.append(p_auc)
    print('AUC for target domain of ' + machine + ': ' + str(auc * 100))
    print('pAUC for target domain of ' + machine + ': ' + str(p_auc * 100))
    
    print('####################################################')
    print('####################################################')
    print('####################################################')

mean_auc = hmean(aucs)
print('mean AUC: ' + str(mean_auc * 100))
mean_p_auc = hmean(p_aucs)
print('mean pAUC: ' + str(mean_p_auc * 100))  
mean_auc_source = hmean(aucs_source)
print('mean AUC for source domain: ' + str(mean_auc_source * 100))
mean_p_auc_source = hmean(p_aucs_source)
print('mean pAUC for source domain: ' + str(mean_p_auc_source * 100))
mean_auc_target = hmean(aucs_target)
print('mean AUC for target domain: ' + str(mean_auc_target * 100))
mean_p_auc_target = hmean(p_aucs_target)
print('mean pAUC for target domain: ' + str(mean_p_auc_target * 100))

score = hmean(aucs_source + aucs_target + p_aucs)
print('final score : ' + str(score*100))

AUC of gearbox: 62.53999999999999
pAUC of gearbox: 52.421052631578945
####################################################
AUC for source domain of gearbox: 55.12000000000001
pAUC for source domain of gearbox: 51.1578947368421
####################################################
AUC for target domain of gearbox: 69.48
pAUC for target domain of gearbox: 54.736842105263165
####################################################
####################################################
####################################################
AUC of ToyTrain: 56.419999999999995
pAUC of ToyTrain: 51.578947368421055
####################################################
AUC for source domain of ToyTrain: 54.36000000000001
pAUC for source domain of ToyTrain: 52.21052631578947
####################################################
AUC for target domain of ToyTrain: 58.60000000000001
pAUC for target domain of ToyTrain: 51.368421052631575
####################################################
####################